In [4]:
import math
from typing import List, Tuple
import numpy as np
import pandas as pd

In [5]:
SAMPLE_PATH = "sample_submission.csv"
OUT_PATH = "submission.csv"

MIN_XY, MAX_XY = -100.0, 100.0
DEFAULT_DEG = 0.0

SMALL_MAX_PUZZLE = 9
HEX_STEP_SMALL = 0.85
HEX_STEP_BIG = 0.8

X_OFF_SMALL = 0.0
Y_OFF_SMALL = 0.0

In [6]:
def sformat(v: float) -> str:
    return "s" + format(float(v), ".9f").rstrip("0").rstrip(".")

In [7]:
def parse_id(id_str: str) -> Tuple[int, int]:
    p, t = id_str.split("_")
    return int(p), int(t)

In [8]:
def generate_hex_points_square_aware(step: float, min_xy: float, max_xy: float, x_off: float = 0.0, y_off: float = 0.0):
    dy = step * math.sqrt(3) / 2.0
    ys = np.arange(min_xy + y_off, max_xy + 1e-9, dy)

    pts = []
    for row_idx, y in enumerate(ys):
        # keep the usual alternating shift, but also apply x_off
        x_shift = 0.0 if (row_idx % 2 == 0) else (step / 2.0)
        xs = np.arange(min_xy + x_off, max_xy + 1e-9, step) + x_shift
        xs = xs[(xs >= min_xy) & (xs <= max_xy)]
        if y < min_xy or y > max_xy:
            continue
        for x in xs:
            pts.append((float(x), float(y)))

    pts = np.array(pts, dtype=float)
    cheb = np.maximum(np.abs(pts[:, 0]), np.abs(pts[:, 1]))
    eucl2 = pts[:, 0] ** 2 + pts[:, 1] ** 2
    order = np.lexsort((eucl2, cheb))
    pts = pts[order]
    return [(float(x), float(y)) for x, y in pts]

In [9]:
def main():
    df = pd.read_csv(SAMPLE_PATH)

    pts_small = generate_hex_points_square_aware(HEX_STEP_SMALL, MIN_XY, MAX_XY, X_OFF_SMALL, Y_OFF_SMALL)
    pts_big = generate_hex_points_square_aware(HEX_STEP_BIG, MIN_XY, MAX_XY, 0.0, 0.0)

    xs_out, ys_out, deg_out = [], [], []

    for _id in df["id"].tolist():
        puzzle_id, tree_idx = parse_id(_id)
        pts = pts_small if puzzle_id <= SMALL_MAX_PUZZLE else pts_big

        x, y = pts[tree_idx]
        x = min(max(x, MIN_XY), MAX_XY)
        y = min(max(y, MIN_XY), MAX_XY)

        xs_out.append(sformat(x))
        ys_out.append(sformat(y))
        deg_out.append(sformat(DEFAULT_DEG))

    out = pd.DataFrame({"id": df["id"], "x": xs_out, "y": ys_out, "deg": deg_out})
    out.to_csv(OUT_PATH, index=False)
    print(f"Saved: {OUT_PATH} (rows={len(out)})")
    print(out.head(12))


if __name__ == "__main__":
    main()


Saved: submission.csv (rows=20100)
       id        x              y deg
0   001_0     s0.3   s0.112536677  s0
1   002_0     s0.3   s0.112536677  s0
2   002_1   s-0.55   s0.112536677  s0
3   003_0     s0.3   s0.112536677  s0
4   003_1   s-0.55   s0.112536677  s0
5   003_2  s-0.125  s-0.623584916  s0
6   004_0     s0.3   s0.112536677  s0
7   004_1   s-0.55   s0.112536677  s0
8   004_2  s-0.125  s-0.623584916  s0
9   004_3   s0.725  s-0.623584916  s0
10  005_0     s0.3   s0.112536677  s0
11  005_1   s-0.55   s0.112536677  s0
